# Travail pratique 1 - IFT 3700
##### Remis le 22 novembre 2020 par Rym Bach et Laurier Lavoie-Giasson
## Introduction
Dans le cadre de ce travail, on s'intéresse à la précision des différentes techniques de classification vues en classe, et de comparer son niveau en utilisant la distance euclidienne ainsi que [...]
## Code
### Installation des packages nécessaires

In [19]:
import sys
!{sys.executable} -m pip install tqdm scikit-image

    100% |████████████████████████████████| 12.5MB 79kB/s  eta 0:00:01
    100% |████████████████████████████████| 153kB 4.6MB/s eta 0:00:01
    100% |████████████████████████████████| 1.6MB 662kB/s eta 0:00:01
    100% |████████████████████████████████| 3.3MB 325kB/s eta 0:00:01
    100% |████████████████████████████████| 4.4MB 238kB/s eta 0:00:01


### Chargement et prétraitement des jeux de données d'entraînement et de tests

In [6]:
%matplotlib inline
import csv
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA

CONST_N_DIM=784

def preprocessing(aX, an_dimensions=CONST_N_DIM):
    #cette fonction sera utilisée pour le prétraitement des données du jeu de données.
    
    ret_val = np.array([np.round(np.divide(aX[i], 255.0)) for i in tqdm(range(len(aX)))])
    
    #application de PCA pour la réduction de dimensionalité
    #pca = PCA(n_components=an_dimensions)
    #ret_val = pca.fit_transform(X_rounded)
    return ret_val
    
def readMNIST(afilename, an_dimensions=CONST_N_DIM):
    #cette fonction lit le fichier de MNIST à l'emplacement fourni et retourne le jeu de
    #données sur lequel on a appliqué la fonction de prétraitement
    data = open(afilename)
    csv_file = csv.reader(data)
    data_points = [row for row in csv_file] #pour enlever les headers
    data_points.pop(0)
    #valeurs Y de taille 1 (labels)
    print("Extraction des labels pour", afilename)
    Y = np.array([int(data_points[i][0]) for i in tqdm(range(len(data_points)))])
    
    #vecteurs X de taille 784 (arrondis à l'entier le plus près)
    print("Extraction des points de données pour", afilename)
    X = np.array([[int(j) for j in data_points[i][1:]] for i in tqdm(range(len(data_points)))])
    X = X.reshape((len(Y),784))
    
    #on retourne un tuple avec les vecteurs x et les valeurs y
    return (preprocessing(X, an_dimensions),Y)

XY_train = readMNIST('mnist_train.csv', CONST_N_DIM)
X_train = XY_train[0]
Y_train = XY_train[1]

XY_test = readMNIST('mnist_test.csv', CONST_N_DIM)
X_test = XY_test[0]
Y_test = XY_test[1]

  1%|          | 329/60000 [00:00<00:18, 3288.25it/s]

Extraction des labels pour mnist_train.csv
Extraction des points de données pour mnist_train.csv


  3%|▎         | 332/10000 [00:00<00:02, 3318.90it/s]

Extraction des labels pour mnist_test.csv
Extraction des points de données pour mnist_test.csv


100%|██████████| 10000/10000 [00:00<00:00, 42547.43it/s]


### _Sanity Check_
Ici on regarde si les données sont encore "saines", c'est à dire si on peut afficher la première lettre du jeu de données

In [ ]:
plt.imshow(X_train[0].reshape((28,28)), cmap="gray")
plt.show()

### Mise en place de la métrique à utiliser

Soit $X = \begin{pmatrix}x_{11} & ... & x_{1n}\\ ... & ... & ... \\ x_{n1} & ... & x_{nn}\end{pmatrix}$, le caractère en question. On calculera d'abord $M_{00}, M_{01}$ et $M_{10}$.

\begin{align}
    M_{ij} = \sum_{k=1}^{d_1}\sum_{\ell}^{d_2}x_{k\ell}k^i\cdot\ell^j
\end{align}

On calculera alors le centroïde de l'image $\{\bar{x},\bar{y}\} = \{\frac{M_{10}}{M_{00}}, \frac{M_{01}}{M_{00}}\}$, ainsi que les moments centraux $\mu_{00}, \mu_{20}, \mu_{02}, \mu_{11}, \mu_{30}, \mu_{03}, \mu_{21}$ et $\mu_{12}$

\begin{align}
    \mu_{ij} = \sum_{k=1}^{d_1}\sum_{\ell=1}^{d_2}x_{k\ell}\cdot(k-\bar{x})^i\cdot(\ell−\bar{y})^j)
\end{align}

Pour finir, on calculera la similarité des points en comparant la distance euclidienne entre les vecteurs contenant leurs moments invariants $\eta_{20}, \eta_{02}, \eta_{11}, \eta_{30}, \eta_{03}, \eta_{21}$ et $\eta_{12}$.

\begin{align}
    \eta_{ij} = \frac{\mu_{ij}}{\mu_{00}^{(1+\frac{i+j}{2})}}
\end{align}

In [18]:
from skimage.measure import moments, moments_central

#on définit la fonction de similarité à utiliser
def moment_distance(ax_1, ax_2):
    x_1=ax_1.reshape((28,28))
    x_2=ax_2.reshape((28,28))
    m=moments(x_1)
    mu=moments_central(x_1)
    
    mij_differences=np.array([computeMij(ax_1, ij[0], ij[1])-computeMij(ax_2, ij[0], ij[1]) for ij in ij_list])
    dist = np.sqrt(np.sum(np.power(mij_differences,2)))
    return 

On voit que la lettre apparait bel et bien, on peut donc considérer que le jeu de données a été correctement chargé
### Classification en utilisant l'algorithme K-médoïdes

In [ ]:
#TODO Classification KMeans
#en utilisant plusieurs valeurs de K (8,9,10, 11, ...)
#et en calculant le score silhouette à chaque valeur de K

### Classification en utilisant l'algorithme des K plus proches voisins

In [ ]:
from multiprocessing import Process
from multiprocessing.sharedctypes import Array
import math
#on va utiliser ici du multi-processing pour travailler autour du GIL de Python,
#on va démarrer plusieurs processus et travailler sur des variables partagées

def fit_and_score(aX_train, aY_train, aX_test, aY_test, ak, aAccuracy_shared_array):
    
    classifier = KNeighborsClassifier(n_neighbors=ak)
    classifier.fit(aX_train, aY_train)
    aAccuracy_shared_array[(ak-1)/2] = classifier.score(aX_test, aY_test)
    return aAccuracy_shared_array[ak]



def classify_KNN(aX_train, aY_train, aX_test, aY_test, max_neighbors, multi_process=True):
    if multi_process:
        #on crée un tableau partagé de doubles "primitifs" de C pour les résultats
        aX_train_shared=Array('d', aX_train.shape[0]*aX_train.shape[1])
        #on le passe à Numpy pour utilisation à travers l'interface "buffer"
        aX_train_mp=np.frombuffer(aX_train_shared.get_obj()).reshape(aX_train.shape)
        #on copie les valeurs de l'argument dans le tableau partagé
        np.copyto(aX_train_mp, aX_train)
        
        aY_train_shared=Array('d', aY_train.size)
        aY_train_mp=np.frombuffer(aY_train_shared.get_obj())
        np.copyto(aY_train_mp, aY_train)
        
        
        aX_test_shared=Array('d', aX_test.shape[0]*aX_test.shape[1])
        aX_test_mp=np.frombuffer(aX_test_shared.get_obj()).reshape(aX_test.shape)
        np.copyto(aX_test_mp, aX_test)
        
        
        aY_test_shared=Array('d', aY_test.size)
        aY_test_mp=np.frombuffer(aY_test_shared.get_obj())
        np.copyto(aY_test_mp, aY_test)
        
        #initialisation des K impairs
        k_range = np.add(np.multiply(np.array(range(math.floor(max_neighbors/2.0))), 2),1)
        k_range.size
        
        #on crée un tableau partagé de doubles "primitifs" de C pour stocker la précision des modèles
        accuracy_array = Array('d', len(k_range))
        accuracy = np.frombuffer(accuracy_array.get_obj())
        #on initialise les valeurs à zéro
        np.copyto(accuracy, np.zeros(len(k_range)))
        
        processes = []
        
        for k in k_range:
            if k % 2 != 0:
                #spawn un processus pour calculer, qui lira la mémoire partagée
                myprocess=Process(target=fit_and_score, args=(aX_train_mp, aY_train_mp, aX_test_mp, aY_test_mp, k, accuracy))
                processes.append(myprocess)
                myprocess.start()
        for i in tqdm(range(len(processes))):
            myprocess=processes[i]
            #join le processus pour arrêter l'exécution jusqu'à temps que les calculs soient terminés
            myprocess.join()
            del myprocess
        #retourner les paires de K et de précision
        return np.array([np.array(k_range), accuracy])
    else:
        classifier = KNeighborsClassifier(n_neighbors=1)
        classifier.fit(aX_train, aY_train)
        print(classifier.score(aX_test, aY_test))
        #TODO changer ceci ASAP. Dois aller dormir lol
        
    
resultats_KNN = classify_KNN(X_train, Y_train, X_test, Y_test, 15, True)   

In [ ]:
plt.plot(resultats_KNN[0], resultats_KNN[1])
plt.xlabel('k')
plt.ylabel('Précision')
plt.title('Précision de la classification en fonction de k');
plt.xticks(np.arange(min(resultats_KNN[0]), max(resultats_KNN[1])+1, 1.0))
plt.show()

### Classification avec 